In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=30d82520a3137c36b055c17be7b484dc68ee95b9eb2b013ff2da1c68c02dbbe8
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.7MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F24 '''
def F24(X):
    f = bn.F24()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.0257 ,random_state=0 , l1_ratio=0.9795, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 8.05 , C = 787.74 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_244000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_24_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(10, 20):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i-10,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 10
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=287350, Sat Apr 18 05:11:58 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 9.420589921315657e+03 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 9.406677881985343e+03 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [9440.16153323] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [9437.30033945] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 9.406584275763877e+03 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   35    665 9.080726097348455e+03 1.1e+00 1.96e+00  2e+00  2e+00 0:03.4
   77   1463 8.917563489243446e+03 1.2e+00 1.62e+00  2e+00  2e+00 0:07.4
  100   1900 8.778399321685785e+03 1.2e+00 1.48e+00  1e+00  1e+00 0:09.6
  164   3116 7.713525380966246e+03 1.2e+00 8.96e-01  9e-01  9e-01 0:15.7
  200   3800 7.075334951848235e+03 1.2e+00 6.07e-01  6e-01  6e-01 0:19.1
  285   5415 6.198293722171726e+03 1.3e+00 2.43e-01  2e-01  2e-01 0:27.2
  300   5700 6.143410666052350e+03 1.3e+00 2.08e-01  2e-01  2e-01 0:28.6
  400   7600 5.984074042876347e+03 1.3e+00 7.02e-02  7e-02  7e-02 0:38.1
  500   9500 5.968196163677776e+03 1.3e+00 2.37e-02  2e-02  2e-02 0:47.7
  600  11400 5.966283203076856e+03 1.3e+00 8.87e-03  9e-03  9e-03 0:57.7
  700  13300 5.966032503238838e+03 1.3e+00 3.09e-03  3e-03  3e-03 1:07.6
  800  15200 5.966005387470789e+03 1.3e+00 1.05e-03  1e-03  1e-03 1:17.3
  900  17100 5.966001387724422e+03 1.3e+00 3.76e-04

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [9477.91441782] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [10009.71283866] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 9.401957125237634e+03 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2
    3     57 8.937265701886394e+03 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3


...

    3     57 9.328349067222056e+03 1.0e+00 2.33e+00  2e+00  2e+00 0:00.4
   28    532 9.157733354840913e+03 1.1e+00 1.98e+00  2e+00  2e+00 0:03.4
   61   1159 8.764808648707873e+03 1.1e+00 1.72e+00  2e+00  2e+00 0:07.4
  100   1900 8.673040963793144e+03 1.2e+00 1.31e+00  1e+00  1e+00 0:12.2
  151   2869 7.727017742938478e+03 1.2e+00 8.00e-01  8e-01  8e-01 0:18.2
  200   3800 6.755915480632545e+03 1.2e+00 4.79e-01  5e-01  5e-01 0:24.4
  266   5054 6.162953199377724e+03 1.2e+00 2.40e-01  2e-01  2e-01 0:32.4
  300   5700 6.070644596612846e+03 1.2e+00 1.73e-01  2e-01  2e-01 0:36.4
  384   7296 5.981483551084090e+03 1.2e+00 6.86e-02  7e-02  7e-02 0:46.5
  400   7600 5.977721425354278e+03 1.2e+00 5.85e-02  6e-02  6e-02 0:48.5
  500   9500 5.967215157770693e+03 1.2e+00 1.88e-02  2e-02  2e-02 1:00.4
  600  11400 5.966150011035096e+03 1.2e+00 6.22e-03  6e-03  6e-03 1:12.4
  700  13300 5.966017740458768e+03 1.2e+00 2.10e-03  2e-03  2e-03 1:24.2
  800  15200 5.966002829884854e+03 1.2e+00 7.23e-04

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [9477.91441782] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [9910.86501205] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 8.981784721110958e+03 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2
    3     57 8.384094587153084e+03 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [9366.07640964] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   35    665 7.258806434372436e+03 1.1e+00 1.88e+00  2e+00  2e+00 0:03.4
   77   1463 5.613330671397784e+03 1.1e+00 1.47e+00  1e+00  1e+00 0:07.4
  100   1900 4.718620593633604e+03 1.2e+00 1.28e+00  1e+00  1e+00 0:09.6
  163   3097 3.321465960972175e+03 1.2e+00 7.92e-01  8e-01  8e-01 0:15.7
  200   3800 2.791712285189978e+03 1.2e+00 6.09e-01  6e-01  6e-01 0:19.2
  283   5377 2.263737303935449e+03 1.2e+00 2.70e-01  3e-01  3e-01 0:27.3
  300   5700 2.201232548670654e+03 1.2e+00 2.29e-01  2e-01  2e-01 0:28.9
  400   7600 2.086947236775903e+03 1.2e+00 8.58e-02  8e-02  9e-02 0:38.5
  500   9500 2.068755876032095e+03 1.2e+00 3.64e-02  4e-02  4e-02 0:48.0
  600  11400 2.066167923874057e+03 1.2e+00 1.58e-02  2e-02  2e-02 0:57.6
  700  13300 2.065473784323523e+03 1.2e+00 7.50e-03  7e-03  8e-03 1:07.3
  800  15200 2.065357497747181e+03 1.3e+00 3.36e-03  3e-03  3e-03 1:16.8
  900  17100 2.065328772421121e+03 1.3e+00 1.65e-03  2e-03  2e-03 1:26.4
 1000  19000 2.065321162305486e+03 1.4e+00 9.67e-04

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F24_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F24(X_Values.iloc[i,:200])
    SVM_Fun [i] = F24(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F24(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([2661.80667092]), count=array([1])) 0.0003830096066747135
SVM
ModeResult(mode=array([9479.24642411]), count=array([1])) 426.32687469753176
ELN
ModeResult(mode=array([10437.83574684]), count=array([1])) 0.00017661197288503918
